In [2]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from pytz import timezone
from scipy import stats

trading_freq = 10 #10 days trading frequency

def initialize(context):
    context.stocks = [sid(32907), sid(3597), sid(19465), sid(8554)] # Set securities
    context.classifier = RandomForestClassifier() # Use a random forest classifier
    #replace every value in the array except the last one with 1.
    context.prediction = np.ones_like(context.stocks[0:-1]) 
    
    context.day_count = -1
    context.allocation = -np.ones_like(context.stocks[0:-1])

In [3]:
def handle_data(context, data):
    #Trade only once per day
    loc_dt = get_datetime().astimezone(timezone('US/Eastern'))
    if loc_dt.hour == 16 and loc_dt.minute == 0:
        context.day_count += 1
        pass
    else:
        return
    
    #Limit trading frequency
    if context.day_count % trading_freq != 0.0:
        return
    
    prices = history(400,'1d','price').as_matrix(context.stocks)
    #converts prices matrix to z-scores with 1 degree of freedom
    changes_all = stats.zscore(prices, axis=0, ddof=1) 
    #z-score difference for each sector fund and the underlying benchmark SPY
    changes = changes_all[:,0:-1] - np.tile(changes_all[:,-1],(len(context.stocks)-1,1)).T 
    changes = changes > 0
    
    for k in range(len(context.stocks)-1):
        X = np.split(changes[:,k],20)
        Y = X[-1]
        
        context.classifier.fit(X,Y) #generate model
        
        context.prediction[k] = context.classifier.predict(Y)
        
    allocation = context.prediction.astype(float) #set the prediction values to float type
    denom = np.sum(allocation) #sum the prediction values.
    
    if denom != 0.0:
        #if the sum is not zero divide each prediction value by the sum of the prediction values.
        allocation = allocation/np.sum(allocation)
        
    if np.array_equal(context.allocation,allocation):
        return    
    
    context.allocation = allocation
    record(num_secs = np.count_nonzero(allocation))
    
  
    
    for stock, percent in zip(context.stocks[0:-1], allocation):
        order_target_percent(stock, percent)
    